## Connect

In [38]:
# Import libraries
import os
from dotenv import load_dotenv
import json
import weaviate
from weaviate import EmbeddedOptions
from weaviate.classes.config import Configure, Property, DataType
from weaviate.classes.query import MetadataQuery, HybridVector, Move
import warnings

warnings.filterwarnings('ignore')

# Load the environment variables
load_dotenv()

aoai_key=os.environ['AZURE_OPENAI_API_KEY']
aoai_endpoint=os.environ['AZURE_OPENAI_ENDPOINT']
aoai_deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']
aoai_embedding=os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Utility Funtions

In [2]:
# JSON print beautifier
def json_print(data):
    print(json.dumps(data, indent=2))

### Create a Weaviate Embedded DB

In [ ]:
client = weaviate.connect_to_embedded(
    version="1.26.1",  # e.g. version="1.26.5"
    headers={
        "X-OpenAI-BaseURL": aoai_endpoint,
        "X-Azure-Api-Key": aoai_key
    },
)

json_print(client.is_ready())

In [ ]:
# Show Weaviate DB metadata
json_print(client.get_meta())

### Create Collection (EU Destinations)

In [5]:
if client.collections.exists("eudestinations"):
    client.collections.delete("eudestinations")

In [ ]:
client.collections.create(
    "eudestinations",
    vectorizer_config=[
        Configure.NamedVectors.text2vec_azure_openai(
            name="title_vector",            
            resource_name="aoai-airlift-1",
            deployment_id=aoai_embedding,
            base_url=aoai_endpoint
        )
    ],
    # Additional parameters not shown
)

### Read Europe Touristic Destinations

In [ ]:
file_path = '../data/eu_destinations_n.json'

with open(file_path, "r") as file:
    data = file.read()

ds = json.loads(data)
json_print(ds[0])

### Create Embeddings

In [8]:
collection = client.collections.get("eudestinations")

with collection.batch.dynamic() as batch:
    for i, d in enumerate(ds):
        weaviate_obj = {
            "destination": d["Destination"],
            "region": d["Region"],
            "country": d["Country"],
            "category": d["Category"],
            "annualtourists": d["Approximate Annual Tourists"],
            "foods": d["Famous Foods"],
            "language": d["Language"],
            "besttimevisit": d["Best Time to Visit"],
            "costliving": d["Cost of Living"],
            "cultural": d["Cultural Significance"],
            "description": d["Description"]
        }

        # The model provider integration will automatically vectorize the object
        batch.add_object(
            properties=weaviate_obj,
            # vector=vector  # Optionally provide a pre-obtained vector
        )

In [ ]:
# count = client.query.aggregate("eudestinations").with_meta_count().do()
# json_print(count)

### Query a specific Collection Dataset

In [ ]:
collection = client.collections.get("eudestinations")

### Search Patterns

In [ ]:
response = collection.query.fetch_objects(
    limit=1
)
for o in response.objects:
    json_print(o.properties)

In [13]:
response = collection.query.near_text(
    query="Picasso",  # The model provider integration will automatically vectorize the query
    limit=5,
    include_vector=True,
    return_metadata=MetadataQuery(distance=True)
)

for obj in response.objects:
    print(obj.properties)
    print(obj.metadata.distance)
    print(obj.vector)

In [7]:
response = collection.query.near_text(
    query="Picasso",  # The model provider integration will automatically vectorize the query    
    distance=0.20, 
    return_metadata=MetadataQuery(distance=True)
)

for obj in response.objects:
    print(obj.properties)
    print(obj.metadata.distance)

### Sparse and Dense Search

In [ ]:
# Sparse
response = collection.query.bm25(
    query="Sea",
    return_metadata=MetadataQuery(score=True),
    limit=3
)

for o in response.objects:
    json_print(o.properties)
    print(o.metadata.score)

### Hybrid Search (Dense and Sparse)

In [ ]:
response = collection.query.hybrid(
    query="sea",
    return_metadata=MetadataQuery(score=True, explain_score=True),
    limit=3
    )

for o in response.objects:
    json_print(o.properties)
    print(o.metadata.score)
    print(o.metadata.explain_score)

In [ ]:
# An alpha of 1 is a pure vector (dense) search.
# An alpha of 0 is a pure keyword (sparse) search.
response = collection.query.hybrid(
    query="sea",
    alpha=0.7,
    limit=10,
)

for o in response.objects:
    json_print(o.properties)

In [44]:
response = collection.query.hybrid(
    query="sea",
    max_vector_distance=0.4,  # Maximum threshold for the vector search component
    vector=HybridVector.near_text(
        query="Scenic view",
        move_away=Move(force=0.5, concepts=["Volcano"]),
    ),
    alpha=0.75,
    limit=10,
)

In [ ]:
for o in response.objects:
    json_print(o.properties)